In [1]:
#importer les framworks et bibliotheques
import numpy as np
from pyvi import ViTokenizer

In [2]:
#Load files
f = open("dao.txt",'r',encoding = 'utf-8-sig')
contents = f.read() #lire le document
    

In [3]:
#Pre-processing
contents_parsed = contents.lower() #Tout convertir en minuscule
contents_parsed = contents_parsed.replace('\n', '. ') #Changer les caractères de la ligne en ponctuation
contents_parsed = contents_parsed.strip() #Supprimer les espaces excédentaires

In [4]:
print(contents) #imprimer le document avant traitement

Trois penaltys concédés
Neymar pensait avoir scellé l’affaire en ouvrant le score de la tête (63e). Mais comme à Liverpool, deux fois, et Naples dans des matches clés de Ligue des champions, le PSG a encore commis l’erreur dans sa surface de réparation… Et offert aux Guingampais, non pas un ni deux mais bien trois penaltys !

Après le premier échec de Marcus Thuram (61e), Yeni Atito Ngbakoto (81e) et Thuram (90+3e) ont saisi leur chance, et sonné le Parc des princes pris de court par ce scénario glaçant. Et voilà le club francilien à terre pour la première fois de la saison sur ses terres, après vingt matches de rang sans perdre !

« On devait tuer ce match »
« On voulait aller en demi-finale mais on n’a pas réussi, c’était un match un peu bizarre. On aurait pu éviter ces trois penalties avec un peu plus de concentration et de réactivité. On devait tuer ce match, ne pas permettre à cette équipe de revenir », a estimé le milieu du PSG Marco Verratti au micro de Canal+ Sport à l’issue du

In [5]:
print(contents_parsed) #imprimer document apres pre-traitement

trois penaltys concédés. neymar pensait avoir scellé l’affaire en ouvrant le score de la tête (63e). mais comme à liverpool, deux fois, et naples dans des matches clés de ligue des champions, le psg a encore commis l’erreur dans sa surface de réparation… et offert aux guingampais, non pas un ni deux mais bien trois penaltys !. . après le premier échec de marcus thuram (61e), yeni atito ngbakoto (81e) et thuram (90+3e) ont saisi leur chance, et sonné le parc des princes pris de court par ce scénario glaçant. et voilà le club francilien à terre pour la première fois de la saison sur ses terres, après vingt matches de rang sans perdre !. . « on devait tuer ce match ». « on voulait aller en demi-finale mais on n’a pas réussi, c’était un match un peu bizarre. on aurait pu éviter ces trois penalties avec un peu plus de concentration et de réactivité. on devait tuer ce match, ne pas permettre à cette équipe de revenir », a estimé le milieu du psg marco verratti au micro de canal+ sport à l’is

In [6]:
#Utilisez la bibliothèque nltk pour séparer les phrases dans le texte
import nltk 

In [7]:
sentences = nltk.sent_tokenize(contents_parsed) #séparer les phrases dans le texte

In [8]:
print(sentences) #imprimer les phrases

['trois penaltys concédés.', 'neymar pensait avoir scellé l’affaire en ouvrant le score de la tête (63e).', 'mais comme à liverpool, deux fois, et naples dans des matches clés de ligue des champions, le psg a encore commis l’erreur dans sa surface de réparation… et offert aux guingampais, non pas un ni deux mais bien trois penaltys !.', '.', 'après le premier échec de marcus thuram (61e), yeni atito ngbakoto (81e) et thuram (90+3e) ont saisi leur chance, et sonné le parc des princes pris de court par ce scénario glaçant.', 'et voilà le club francilien à terre pour la première fois de la saison sur ses terres, après vingt matches de rang sans perdre !.', '.', '« on devait tuer ce match ».', '« on voulait aller en demi-finale mais on n’a pas réussi, c’était un match un peu bizarre.', 'on aurait pu éviter ces trois penalties avec un peu plus de concentration et de réactivité.', 'on devait tuer ce match, ne pas permettre à cette équipe de revenir », a estimé le milieu du psg marco verratti

In [9]:
#transférer le mot au vecteur
from gensim.models import KeyedVectors 

w2v = KeyedVectors.load_word2vec_format("fr.vec")

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [10]:
vocab = w2v.wv.vocab

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [11]:

X = []
for sentence in sentences:
    sentence_tokenized = ViTokenizer.tokenize(sentence)
    words = sentence_tokenized.split(" ")
    sentence_vec = np.zeros((300))
    for word in words:
        if word in vocab:
            sentence_vec+=w2v.wv[word]
    X.append(sentence_vec)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


In [15]:
#Regroupement Kmeans clustering
from sklearn.cluster import KMeans

n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters)
kmeans = kmeans.fit(X)

In [16]:

from sklearn.metrics import pairwise_distances_argmin_min

avg = []
for j in range(n_clusters):
    idx = np.where(kmeans.labels_ == j)[0]
    avg.append(np.mean(idx))
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, X)
ordering = sorted(range(n_clusters), key=lambda k: avg[k])
summary = ' '.join([sentences[closest[idx]] for idx in ordering])

In [17]:
summary

'on devait tuer ce match, ne pas permettre à cette équipe de revenir », a estimé le milieu du psg marco verratti au micro de canal+ sport à l’issue du match.. . neymar pensait avoir scellé l’affaire en ouvrant le score de la tête (63e). . « c’est difficilement explicable, il y a trois niveaux d’écart, de grands joueurs et un grand entraîneur, mais on est resté calme. même son de cloche du côté, l’entraîneur francilien thomas tuchel : « nous avons eu beaucoup d’occasions pour gagner, on aurait pu marquer trois buts, mais on ne l’a pas fait, honnêtement, on a joué avec trop de confiance.'